In [1]:
import csv
import json
import math as m
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr
import datetime as dt
import yfinance as yf
import seaborn as sb
from datetime import datetime, timedelta
from functools import reduce
import yahoo_fin.stock_info as si
from yahoo_fin import *
import jinja2
import tradingview_ta
from tradingview_ta import TA_Handler, Interval, Exchange
from yahoofinancials import YahooFinancials
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier
from pypfopt import plotting
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import EfficientCVaR
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
plt.style.use('bmh')

In [2]:
# created a csv file of ticker symbols for an excel database
nasdaq = pd.DataFrame(si.tickers_nasdaq())
sym = set( symbol for symbol in nasdaq[0].values.tolist() )

# deleting the inactive tickers by suffix
my_list = ['C', 'W', 'T', 'R', 'P', 'Q', 'Z', 'I', '.A']

# variable for tickers to be saved and disgarded
del_set = set()
sav_set = set()

# loop to id the suffix and disgard
for symbol in sym:
    if len( symbol ) > 4 and symbol[-1] in my_list:
        del_set.add( symbol )
    else:
        sav_set.add( symbol )
        
tickers = sorted(sav_set)

In [5]:
# Use this code if you need to update it on VSCode
yf.pdr_override

# Calculate the date range
end = dt.datetime.today().strftime('%Y-%m-%d')
begin = (dt.datetime.today() - dt.timedelta(days=30)).strftime('%Y-%m-%d')

# Create an empty DataFrame to store the stock prices
prices = pd.DataFrame()

# Iterate through each stock ticker
for stock in tickers:
    try:
        # Download the Adj Close data for each stock
        stock_data = yf.download(stock, start=begin, end=end, progress=False)

        # Check if the DataFrame is empty (no data downloaded)
        if not stock_data.empty:
            # Rename the 'Adj Close' column to the stock ticker
            stock_data.rename(columns={'Adj Close': stock}, inplace=True)

            # Concatenate the stock data to the prices DataFrame
            if prices.empty:
                prices = stock_data[[stock]]
            else:
                prices = pd.concat([prices, stock_data[[stock]]], axis=1)
        else:
            print(f"No data available for {stock}.")
    except Exception as e:
        print(f"An error occurred while fetching {stock}: {str(e)}")

# Display the updated prices DataFrame
prices.dropna()
prices

TypeError: string indices must be integers